In [ ]:
!pip install google-api-python-client
from googleapiclient.discovery import build
import pandas as pd
import seaborn as sns

In [ ]:
api_key = 'API_KEY'
#channel_id = 'UCnz-ZXXER4jOvuED5trXfEA'
channel_ids = ['UCnz-ZXXER4jOvuED5trXfEA', # techTFQ
               'UCLLw7jmFsvfIVaUFsLs8mlQ', # Luke Barousse
               'UCiT9RITQ9PW6BhXK0y2jaeg', # Ken Jee
               'UC7cs8q-gJRlGwj4A80mCmXg', # Alex the analyst
               'UC2UXDak6o7rBm23k3Vv5dww'  # Tina Huang
                ]

youtube = build('youtube', 'v3', developerKey=api_key)

## Function to get channel statistics

In [ ]:
def get_channel_stats(youtube, channel_ids):
    all_data = []
    request = youtube.channels().list(
                part='snippet,contentDetails,statistics',
                id=','.join(channel_ids))
    response = request.execute()

    for i in range(len(response['items'])):
        data = dict(Channel_name = response['items'][i]['snippet']['title'],
                Subscribers = response['items'][i]['statistics']['subscriberCount'],
                Views = response['items'][i]['statistics']['viewCount'],
                Total_videos = response['items'][i]['statistics']['videoCount'],
                playlist_id = response['items'][i]['contentDetails']['relatedPlaylists']['uploads']
                   )
        all_data.append(data)

    return all_data

In [ ]:
channel_statistics = get_channel_stats(youtube, channel_ids)

In [ ]:
channel_data = pd.DataFrame(channel_statistics)

In [ ]:
channel_data

,Channel_name,Subscribers,Views,Total_videos,playlist_id
0,Ken Jee,271000,9438550,288,UUiT9RITQ9PW6BhXK0y2jaeg
1,Tina Huang,892000,39372005,255,UU2UXDak6o7rBm23k3Vv5dww
2,Luke Barousse,530000,26444690,165,UULLw7jmFsvfIVaUFsLs8mlQ
3,techTFQ,371000,21448499,140,UUnz-ZXXER4jOvuED5trXfEA


In [ ]:
# playlist_id = channel_data.loc[channel_data['Channel_name']=='Ken Jee', 'playlist_id'].iloc[0]

In [ ]:
all_playlist_ids = channel_data['playlist_id'].tolist()

In [ ]:
channel_data['Subscribers'] = pd.to_numeric(channel_data['Subscribers'])
channel_data['Views'] = pd.to_numeric(channel_data['Views'])
channel_data['Total_videos'] = pd.to_numeric(channel_data['Total_videos'])

## Function to get video ids

In [ ]:
def get_video_ids(youtube, playlist_id):

    request = youtube.playlistItems().list(
                part='contentDetails',
                playlistId = playlist_id,
                maxResults = 50)

    response = request.execute()

    video_ids = []

    for i in range(len(response['items'])):
        video_ids.append(response['items'][i]['contentDetails']['videoId'])

    next_page_token = response.get('nextPageToken')
    more_pages = True

    while more_pages:
        if next_page_token is None:
            more_pages = False
        else:
            request = youtube.playlistItems().list(
                part='contentDetails',
                playlistId = playlist_id,
                maxResults = 50,
                pageToken = next_page_token)
            response = request.execute()

            for i in range(len(response['items'])):
                video_ids.append(response['items'][i]['contentDetails']['videoId'])

            next_page_token = response.get('nextPageToken')


    return video_ids

In [ ]:
all_video_ids = []

for playlist_id in all_playlist_ids:
    video_ids = get_video_ids(youtube, playlist_id)
    all_video_ids.extend(video_ids)

print(f"Total video IDs collected: {len(all_video_ids)}")

Total video IDs collected: 847


## Function to get video details

In [ ]:
def get_video_details(youtube, video_ids):
    all_video_stats = []

    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
                    part = 'snippet,statistics',
                    id = ','.join(video_ids[i:i+50]))
        response = request.execute()

        for video in response['items']:
            video_stats = dict(Title = video['snippet']['title'],
                               Published_date = video['snippet']['publishedAt'],
                               Views = video['statistics']['viewCount'],
                               Likes = video['statistics']['likeCount'],
                               Comments = video['statistics']['commentCount']
                              )
            all_video_stats.append(video_stats)

    return all_video_stats

In [ ]:
def collect_all_video_details(youtube, channel_data):
  all_video_details = []

  for i in range(len(channel_data)):
      playlist_id = channel_data.loc[i, 'playlist_id']
      channel_name = channel_data.loc[i, 'Channel_name']

      video_ids = get_video_ids(youtube, playlist_id)
      video_details = get_video_details(youtube, video_ids)

      for v in video_details:
          v['Channel_name'] = channel_name

      all_video_details.extend(video_details)

  return all_video_details

In [ ]:
all_video_details = collect_all_video_details(youtube, channel_data)

In [ ]:
video_data = pd.DataFrame(all_video_details)

In [ ]:
video_data.to_csv("youtube_video_data_cleaned.csv", index=False)